In [1]:
from pathlib import Path
import pandas as pd

def criar_df_estabelecimento(caminho_arquivo: Path):
    # carregando o dataframe estabelecimento
    # definindo as colunas que serão carregadas
    colunas_selecionadas = [0, 1, 2, 3, 4, 5, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 27]
    # definindo o nome das colunas
    colunas_estabelecimentos = ['CNPJ BÁSICO',
                                'CNPJ ORDEM',
                                'CNPJ DV',
                                'ID MATRIZ/FILIAL',
                                'NOME FANTASIA',
                                'SITUAÇÃO CADASTRAL',
                                'DATA INÍCIO ATIVIDADE',
                                'CNAE PRINCIPAL',
                                'CNAE SECUNDÁRIA',
                                'TIPO DE LOGRADOURO',
                                'LOGRADOURO',
                                'NÚMERO',
                                'COMPLEMENTO',
                                'BAIRRO',
                                'CEP',
                                'UF',
                                'COD_MUNICÍPIO',
                                'DDD 1',
                                'TELEFONE 1',
                                'CORREIO ELETRÔNICO']
    df_estabelecimento = pd.read_csv(
        caminho_arquivo,
        sep=';',
        encoding='latin-1',
        usecols=colunas_selecionadas,
        names=colunas_estabelecimentos,
        parse_dates=['DATA INÍCIO ATIVIDADE'],
        dtype={'CEP': 'string', 'DDD 1': 'string', 'TELEFONE 1': 'string', 'CNPJ ORDEM': 'string', 'CNPJ DV': 'string', 'CNPJ BÁSICO': 'string'},
        )
    # selecionando as empresas ativas
    df_empresas_ativos = df_estabelecimento[df_estabelecimento['SITUAÇÃO CADASTRAL'] == 2]

    # selecionando apenas as empresas do RJ
    df_empresas_ativos_rj = df_empresas_ativos[df_empresas_ativos['UF'] == 'RJ'].copy()

    # excluindo as colunas UF e SITUAÇÃO CADASTRAL
    df_empresas_ativos_rj.drop(columns=['SITUAÇÃO CADASTRAL', 'UF'], inplace=True)

    # filtrando para ter apenas empresas com telefone ou e-mail
    df_empresas_ativos_rj_filtrado = df_empresas_ativos_rj[~(df_empresas_ativos_rj['TELEFONE 1'].notnull() & df_empresas_ativos_rj['CORREIO ELETRÔNICO'].notnull())]
    
    return df_empresas_ativos_rj_filtrado
    

In [2]:
def criar_df_municipio(caminho_arquivo: Path):
    # criando um dataframe contendo os códigos e seus respectivos nomes
    dataframe_municipios = pd.read_csv(caminho_arquivo, sep=';', encoding='latin-1', names=['COD_MUNICÍPIO', 'MUNICÍPIO'], low_memory=True)
    return dataframe_municipios

In [4]:
def criar_df_empresa(caminho_arquivo: Path):
    # carregando o dataframe empresas0
    # definindo o nome das colunas
    colunas_empresas =  ['CNPJ BÁSICO', 'RAZÃO SOCIAL','CAPITAL SOCIAL', 'PORTE']
    df_empresas = pd.read_csv(
        caminho_arquivo, sep=';', encoding='latin-1', usecols=[0, 1, 4, 5], names=colunas_empresas, dtype={'CNPJ BÁSICO': 'string', 'PORTE': 'Int64'}
        )
    # Definindo o padrão dos nomes dos MEIs
    padrao_regex = r'\d{2}\.\d{3}\.\d{3}|\d{11}'
    # Filtrando apenas as empresas que não são MEIs
    df_filtrado = df_empresas[~df_empresas['RAZÃO SOCIAL'].str.contains(padrao_regex, regex=True, na=False)]

    return df_filtrado

In [6]:
# obter lista de dataframe empresas
# obter lista de dataframe estabelecimentos
def obter_lista_dataframe(caminho: Path, nome_tipo: str):
    # percorrendo pela pasta e salvando todos os dataframes em uma lista
    lista_dataframe = []
    for pasta in caminho.iterdir():
        if nome_tipo in pasta.name:
            # obter o nome do arquivo dentro da pasta
            for arquivo in pasta.iterdir():
                if arquivo:
                    # aplicar a função
                    if nome_tipo == 'Estabelecimento':
                        dataframe = criar_df_estabelecimento(arquivo)
                    else:
                        dataframe = criar_df_empresa(arquivo)
                    lista_dataframe.append(dataframe)

    # Concatendo todos os dataframes encontrados
    dataframe_concatenado = pd.concat(lista_dataframe, ignore_index=True)
    return dataframe_concatenado

In [24]:
from typing import List

def criar_df_final(lista_dataframe_estabelecimento: List[pd.DataFrame], lista_dataframe_empresa: List[pd.DataFrame], dataframe_municipios: pd.DataFrame):
    df_merge = pd.merge(lista_dataframe_estabelecimento, lista_dataframe_empresa, on='CNPJ BÁSICO', how='inner')
    df_merge_final = pd.merge(df_merge, dataframe_municipios, on='COD_MUNICÍPIO')
    df_merge_final['CNPJ'] = df_merge_final['CNPJ BÁSICO'] + df_merge_final['CNPJ ORDEM'] + df_merge_final['CNPJ DV']
    df_merge_final.drop(columns=['CNPJ BÁSICO', 'CNPJ ORDEM', 'CNPJ DV', 'COD_MUNICÍPIO'], inplace=True)
    return df_merge_final

In [7]:
dataframe_estabelecimento = obter_lista_dataframe(Path(r'Baixados\Selecionados'), 'Estabelecimento')

In [9]:
dataframe_empresa = obter_lista_dataframe(Path(r'Baixados\Selecionados'), 'Empresa')

In [10]:
dataframe_municipios = criar_df_municipio(Path(r'Baixados\Selecionados\Municipios\F.K03200$Z.D50307.MUNICCSV'))

In [11]:
df_merge = pd.merge(dataframe_estabelecimento, dataframe_empresa, on='CNPJ BÁSICO', how='inner')

In [14]:
df_merge_final = pd.merge(df_merge, dataframe_municipios, on='COD_MUNICÍPIO')

In [16]:
df_merge_final['CNPJ'] = df_merge_final['CNPJ BÁSICO'] + df_merge_final['CNPJ ORDEM'] + df_merge_final['CNPJ DV']

In [17]:
df_merge_final.drop(columns=['CNPJ BÁSICO', 'CNPJ ORDEM', 'CNPJ DV', 'COD_MUNICÍPIO'], inplace=True)

In [12]:
df_merge_final.to_excel('Empresas_ativas_rj.xlsx', index=False)

In [25]:
df_final = criar_df_final(dataframe_estabelecimento, dataframe_empresa, dataframe_municipios)

In [27]:
df_final.head(5)

,ID MATRIZ/FILIAL,NOME FANTASIA,DATA INÍCIO ATIVIDADE,CNAE PRINCIPAL,CNAE SECUNDÁRIA,TIPO DE LOGRADOURO,LOGRADOURO,NÚMERO,COMPLEMENTO,BAIRRO,CEP,DDD 1,TELEFONE 1,CORREIO ELETRÔNICO,RAZÃO SOCIAL,CAPITAL SOCIAL,PORTE,MUNICÍPIO,CNPJ
0,1,NaN,2018-02-27 00:00:00,8299704,NaN,TRAVESSA,DO PACO,00023,GRP 812,CENTRO,20010170,21,30130276,NaN,LEONARDO SCHULMANN - LEILOEIRO PUBLICO,"10000,00",5,RIO DE JANEIRO,29799852000172
1,1,NaN,2018-02-22 00:00:00,6470101,NaN,AVENIDA,REPUBLICA DO CHILE,330,TORRE OESTE 14 ANDAR,CENTRO,20031170,21,32192500,NaN,ROOT GENERAL FUNDO DE INVESTIMENTO EM DIREITOS...,"0,00",5,RIO DE JANEIRO,29800237000139
2,1,NaN,2018-02-28 00:00:00,4634603,4930202,AVENIDA,BRASIL,19001,PAVILHAO 12,IRAJA,21530900,21,37083352,NaN,JUDA COMERCIO DE PESCADOS LTDA,"96000,00",1,RIO DE JANEIRO,29800297000151
3,2,OTICA BOA VISTA,2018-02-28 00:00:00,4774100,NaN,LARGO,NOVE DE ABRIL,27,SALA 702 EDIF CECISA II,VILA SANTA CECILIA,27260180,24,99910413,NaN,DORA FILHO OTICA LTDA,"85000,00",3,VOLTA REDONDA,32161267000580
4,1,NaN,2017-06-29 00:00:00,4645101,"4645102,4645103,4646001,4646002,4649408,4664800",RUA,DOUTOR OLIVEIRA BOTELHO,09,SALA 507,SAO FRANCISCO,24360500,21,27199409,NaN,IMPFIX COMERCIO DE MATERIAIS HOSPITALARES LTDA,"94000,00",1,NITEROI,28072879000104
